In [ ]:
from datasets import load_dataset

ds = load_dataset("oliverj990/clock-faces-v1-times")
dataset = ds["train"]

In [ ]:
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image
from openai import OpenAI
from io import BytesIO
import base64

client = OpenAI(api_key="your/api-key")



In [ ]:
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm


model_raw_name = 'gpt4o'

prompt = '''What time is shown on the clock in the given image?'''

output_path = f'Results/real_clocks/Test_{model_raw_name}_FT_Results.xlsx'
df = pd.DataFrame(columns=['answer', f'answer-{model_raw_name}'])


def pil_image_to_base64(image: Image.Image) -> str:
    buffered = BytesIO()
    image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


for index, row in tqdm(enumerate(dataset), total = len(dataset)):
    if os.path.exists(output_path):
        continue
    image = row['image']
    image = image.convert("RGB")
    base64_image = pil_image_to_base64(image)
    response = client.chat.completions.create(
            model='gpt-4.1-2025-04-14',
            messages=[
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": prompt},
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ],
            max_tokens=500,
            temperature=0,
    )
    output_text = response.choices[0].message.content
    # print(output_text)
    
    df.at[index, 'answer'] = row['time']
    df.at[index, f'answer-{model_raw_name}'] = output_text
    # print(decoded)
        
df.to_excel(output_path, index=False, engine='openpyxl')